In [106]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder.appName('abc').getOrCreate()

In [2]:
df = spark.read.option("inferSchema","true").csv("pc2/anime_cleaned.csv",header=True)

In [3]:
df.printSchema()

root
 |-- anime_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- title_english: string (nullable = true)
 |-- title_japanese: string (nullable = true)
 |-- title_synonyms: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- type: string (nullable = true)
 |-- source: string (nullable = true)
 |-- episodes: string (nullable = true)
 |-- status: string (nullable = true)
 |-- airing: string (nullable = true)
 |-- aired_string: string (nullable = true)
 |-- aired: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- score: string (nullable = true)
 |-- scored_by: string (nullable = true)
 |-- rank: double (nullable = true)
 |-- popularity: double (nullable = true)
 |-- members: double (nullable = true)
 |-- favorites: integer (nullable = true)
 |-- background: string (nullable = true)
 |-- premiered: string (nullable = true)
 |-- broadcast: string (nullable = true)
 |-- related: string (nullabl

In [6]:
df2 = spark.read.option("inferSchema","true").csv("pc2/users_cleaned.csv",header=True)

In [7]:
df2.printSchema()

root
 |-- username: string (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_watching: integer (nullable = true)
 |-- user_completed: integer (nullable = true)
 |-- user_onhold: integer (nullable = true)
 |-- user_dropped: integer (nullable = true)
 |-- user_plantowatch: integer (nullable = true)
 |-- user_days_spent_watching: double (nullable = true)
 |-- gender: string (nullable = true)
 |-- location: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- access_rank: string (nullable = true)
 |-- join_date: timestamp (nullable = true)
 |-- last_online: timestamp (nullable = true)
 |-- stats_mean_score: string (nullable = true)
 |-- stats_rewatched: string (nullable = true)
 |-- stats_episodes: double (nullable = true)



### Analisis del csv de animes

In [21]:
df.select("title","title_english").show()

+--------------------+--------------------+
|               title|       title_english|
+--------------------+--------------------+
|       Inu x Boku SS|Inu X Boku Secret...|
|    Seto no Hanayome|My Bride is a Mer...|
|  Shugo Chara!! Doki|  Shugo Chara!! Doki|
|       Princess Tutu|       Princess Tutu|
| Bakuman. 3rd Season|            Bakuman.|
| Yume-iro Pâtissière|                null|
|        Ultra Maniac|Ultramaniac - Mag...|
|Shakugan no Shana...|Shakugan no Shana...|
|Nodame Cantabile:...|                null|
|Ouran Koukou Host...|Ouran High School...|
|  Junjou Romantica 2|   Junjo Romantica 2|
|Kaichou wa Maid-s...|          Maid Sama!|
|Sekaiichi Hatsukoi 2|Sekai Ichi Hatsuk...|
|Tonari no Kaibuts...|   My Little Monster|
|              Bleach|              Bleach|
|             Chobits|             Chobits|
|      Kimi ni Todoke|Kimi ni Todoke: F...|
|  Naruto: Shippuuden|   Naruto: Shippuden|
|             Ranma ½|             Ranma ½|
|           Toradora!|          

In [123]:
df.select("episodes").agg(F.count("episodes")).show()
df.select("title","episodes").where(((F.col("episodes") > 0) & (F.col("episodes") < 24)))\
    .agg(F.count("episodes").alias("0 - 24")).withColumn("Porcentaje",(F.col("0 - 24")/6668)*100).show()
df.select("title","episodes").where(((F.col("episodes") > 25) & (F.col("episodes") < 100)))\
    .agg(F.count("episodes").alias("25 - 100")).withColumn("Porcentaje",(F.col("25 - 100")/6668)*100).show()
df.select("title","episodes").where(((F.col("episodes") > 101) & (F.col("episodes") < 200)))\
    .agg(F.count("episodes").alias("101 - 200")).withColumn("Porcentaje",(F.col("101 - 200")/6668)*100).show()
df.select("title","episodes").where(((F.col("episodes") > 201) & (F.col("episodes") < 300)))\
    .agg(F.count("episodes").alias("200 +")).withColumn("Porcentaje",(F.col("200 +")/6668)*100).show()

+---------------+
|count(episodes)|
+---------------+
|           6668|
+---------------+

+------+-----------------+
|0 - 24|       Porcentaje|
+------+-----------------+
|  5178|77.65446910617877|
+------+-----------------+

+--------+------------------+
|25 - 100|        Porcentaje|
+--------+------------------+
|     928|13.917216556688663|
+--------+------------------+

+---------+------------------+
|101 - 200|        Porcentaje|
+---------+------------------+
|       63|0.9448110377924415|
+---------+------------------+

+-----+------------------+
|200 +|        Porcentaje|
+-----+------------------+
|   12|0.1799640071985603|
+-----+------------------+



In [121]:
df.select("airing").agg(F.count("airing")).show()
df.select("title","airing").where(F.col("airing") == "True")\
    .agg(F.count("airing").alias("En vivo")).withColumn("Porcentaje",(F.col("En vivo")/6668)*100).show()
df.select("title","airing").where(F.col("airing") == "False")\
    .agg(F.count("airing").alias("No en vivo")).withColumn("Porcentaje",(F.col("No en vivo")/6668)*100).show()

+-------------+
|count(airing)|
+-------------+
|         6668|
+-------------+

+-------+------------------+
|En vivo|        Porcentaje|
+-------+------------------+
|    183|2.7444511097780446|
+-------+------------------+

+----------+-----------------+
|No en vivo|       Porcentaje|
+----------+-----------------+
|      6481|97.19556088782244|
+----------+-----------------+



In [133]:
df.select("score","rank","scored_by").where(F.col("score") <= 9.25).orderBy("score",ascending=False).show()
df.select("title","score").where(F.col("score") > 9.0)\
    .agg(F.count("score").alias("score")).withColumn("Porcentaje",(F.col("score")/6668)*100).show()
df.select("title","score").where(F.col("score") < 9.0)\
    .agg(F.count("score").alias("score")).withColumn("Porcentaje",(F.col("score")/6668)*100).show()

+-----+----+---------+
|score|rank|scored_by|
+-----+----+---------+
| 9.25| 1.0|   733592|
| 9.19| 2.0|   471398|
| 9.15| 3.0|    71751|
| 9.15| 4.0|    26739|
| 9.14| 5.0|   563857|
| 9.11| 8.0|   403377|
| 9.11| 7.0|    92025|
| 9.11| 6.0|    29036|
|  9.1| 9.0|    30779|
| 9.07|10.0|    63468|
| 9.04|11.0|    61010|
| 9.04|12.0|   296261|
| 9.02|13.0|    42842|
| 9.02|13.0|    40441|
| 9.01|15.0|   141830|
| 9.01|14.0|   363239|
| 8.99|17.0|   157934|
| 8.95|18.0|   543904|
| 8.92|19.0|   498602|
| 8.91|20.0|   166905|
+-----+----+---------+
only showing top 20 rows

+-----+------------------+
|score|        Porcentaje|
+-----+------------------+
|   17|0.2549490101979604|
+-----+------------------+

+-----+-----------------+
|score|       Porcentaje|
+-----+-----------------+
| 6647|99.68506298740252|
+-----+-----------------+

